## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is, essentially, the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

* Describe RDDs as fundamental storage units in the Spark computing environment
* Create RDDs from Python collections
* Set number of partitions for parallelizing RDDs
* Review an RDD's dependency graph at different stages of processing. 
* Apply the map(func) transformation to a given function on all elements of an RDD in different partitions
* Use collect() action to trigger the processing stage of spark's lazy evaluation
* Use count() action to calculate the number of elements of a parallelized RDD
* Use filter(func) to filter unwanted data from RDDs
* Develop an understanding of Python's lambda functions for RDDs processing


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies will performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified.
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the Database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new data set from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base data set (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the Driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some of key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = list(range(1000))
len(data)

#1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use 'local[*]' as the master.

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `parallelize` method to create an rdd will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [3]:
rdd = sc.parallelize(data, 10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the getNumPartitions method.

In [4]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first n items in the RDD
* `top`: returns the top n items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [7]:
# first
rdd.first()

0

In [9]:
# take
rdd.take(5)

[0, 1, 2, 3, 4]

In [10]:
# top
rdd.top(5)

[999, 998, 997, 996, 995]

In [11]:
# collect
rdd.collect()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 be a random number from 0-1.

In [12]:
import random
import numpy as np

nums = np.array(range(1,1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([3.94551635e-01, 1.15226138e+00, 6.72373438e-01, 3.02181752e+00,
       2.90454568e+00, 8.41560304e-01, 3.58590875e+00, 2.58858848e+00,
       2.87573592e+00, 5.47465054e+00, 5.79401243e+00, 9.32644700e+00,
       5.31932890e+00, 6.83422150e+00, 1.19535024e+01, 5.64605978e+00,
       7.37919785e+00, 1.48289895e+01, 9.19067150e+00, 3.99985384e+00,
       1.34438034e+01, 1.70790428e+01, 1.19885582e+01, 3.90468717e-01,
       1.41045538e+00, 2.11980101e+01, 1.85375112e+01, 1.24957726e+00,
       1.55728311e+01, 6.80167177e+00, 1.98361828e+01, 3.04401081e+00,
       1.82492370e+01, 3.33921908e+01, 3.24937662e+01, 1.46045885e+01,
       6.18980611e+00, 3.13061655e+01, 8.37743363e+00, 3.16482804e+01,
       1.44234973e+01, 5.68129425e+00, 1.57502058e+00, 2.56607226e+01,
       4.98603423e+00, 3.78236047e+00, 1.74130758e+01, 6.48053016e+00,
       1.71903547e+01, 6.84202671e+00, 2.77115654e+01, 7.64180348e-02,
       3.54950717e+00, 6.29450022e+00, 4.80909295e+01, 1.27931030e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [13]:
price_items = sc.parallelize(sales_figures, 10)


Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax` that returns the amount of money our company will receive after the sale tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the map method and assign it to a variable `renenue_minus_tax`

In [15]:
def sales_tax(num):
    return num*.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [16]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.36298750458514856,
 1.0600804730931253,
 0.6185835628005315,
 2.7800721227980936,
 2.67218202753705,
 0.7742354797311995,
 3.2990360503125724,
 2.381501403970284,
 2.6456770467120214,
 5.036678493424304]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a map method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [17]:
discounted = revenue_minus_tax.map(lambda x: x*0.9)

In [18]:
discounted.take(10)

[0.32668875412663373,
 0.9540724257838128,
 0.5567252065204784,
 2.502064910518284,
 2.4049638247833447,
 0.6968119317580797,
 2.969132445281315,
 2.1433512635732557,
 2.381109342040819,
 4.533010644081874]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff use a collection method to see the 15 costliest items.

In [19]:
price_items.map(sales_tax).map(lambda x: x*0.9).top(15)

[804.5151660469808,
 786.0468408691427,
 770.8160662992331,
 769.6244187033906,
 757.4153851025328,
 756.9403710152579,
 753.5896407310317,
 753.1479557780955,
 740.9806326054932,
 732.8017527867266,
 732.2124327562597,
 731.0770834286665,
 725.9053180840152,
 721.8723810017699,
 715.1632524349053]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [20]:
discounted.toDebugString()

b'(10) PythonRDD[9] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use flatMap rather than a simple map. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will a tuple with (original price, post-discount price).

In [21]:
mapped = price_items.map(lambda x:(x, x*0.92*0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.3945516354186397, 0.32668875412663373), (1.1522613837968751, 0.9540724257838128), (0.6723734378266647, 0.5567252065204784), (3.0218175247805363, 2.502064910518284), (2.9045456821054887, 2.4049638247833447), (0.8415603040556516, 0.6968119317580797), (3.5859087503397524, 2.969132445281315), (2.5885884825763954, 2.1433512635732557), (2.8757359203391535, 2.381109342040819), (5.474650536330765, 4.533010644081874)]


Note that we have 1000 tuples created to our specification. Let's take a look at how flatMap differs in its implementation. Use the `flatMap` method with the same function you created above.

In [22]:
flat_mapped = price_items.flatMap(lambda x:(x, x*0.92*0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.3945516354186397, 0.32668875412663373, 1.1522613837968751, 0.9540724257838128, 0.6723734378266647, 0.5567252065204784, 3.0218175247805363, 2.502064910518284, 2.9045456821054887, 2.4049638247833447]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use flatmap rather than map in order to properly reduce to our specifications. This is not one of those instances, but int he upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the filter method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a filter function to meet the consultant's suggestion's specifications. set RDD = `selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [23]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)

# calculate total remaining in inventory 
selected_items.count()

268

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use a reduce method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [25]:
selected_items.reduce(lambda x,y: x + y)

124304.03237397692

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [26]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1,50),x))

sales_data.take(7)

[(33, 311.85936717656057),
 (35, 316.63181252720386),
 (11, 305.15321258886587),
 (39, 325.28355467381283),
 (42, 327.67548752410886),
 (19, 314.30578177524563),
 (30, 305.2158643902881)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called __reduceByKey__ to perform reducing operations while grouping by keys. After you have calculated the total, use the __sortBy__ method on the RDD to rank the users from the highest spending to the least spending.



In [27]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x,y: x+y)
total_spent.take(10)

[(50, 3209.833897809516),
 (40, 1034.0198024613915),
 (30, 2797.3172003263962),
 (20, 2739.7500247496664),
 (10, 2308.1438015680014),
 (11, 2772.826613986864),
 (21, 2333.4573817618057),
 (31, 1647.6878096823004),
 (41, 1805.3683810021985),
 (1, 1654.7737482098805)]

In [28]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1], ascending=False).collect()

[(39, 5103.415908431285),
 (36, 4943.951940734657),
 (43, 4433.280066416479),
 (24, 4201.8651322830065),
 (35, 4134.826171909113),
 (27, 3934.912518253257),
 (37, 3875.570218166396),
 (9, 3668.503577897419),
 (5, 3399.7294009854318),
 (8, 3258.6105598869303),
 (50, 3209.833897809516),
 (13, 3159.254384285821),
 (44, 3066.8454276541925),
 (33, 2970.2406576667927),
 (34, 2863.5028692634323),
 (25, 2809.801665747578),
 (30, 2797.3172003263962),
 (11, 2772.826613986864),
 (16, 2740.369586490811),
 (20, 2739.7500247496664),
 (3, 2607.447192221106),
 (28, 2491.3854524885783),
 (19, 2347.7759766897425),
 (21, 2333.4573817618057),
 (10, 2308.1438015680014),
 (22, 2302.842469607832),
 (26, 2252.893410342373),
 (2, 2229.1488654527275),
 (12, 2225.0812865089083),
 (42, 2203.6714500741464),
 (14, 2135.9741534499985),
 (29, 2094.5920381899587),
 (15, 2073.0850922133686),
 (38, 1910.2461772586346),
 (45, 1904.7841624596063),
 (4, 1902.1781857561184),
 (23, 1886.9329828979921),
 (41, 1805.36838100219

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [29]:
total_items = sales_data.countByKey()
sorted(total_items.items(), key=lambda kv: kv[1], reverse=True)

[(22, 13),
 (15, 10),
 (5, 9),
 (40, 9),
 (3, 9),
 (46, 9),
 (36, 8),
 (41, 8),
 (47, 8),
 (9, 8),
 (16, 7),
 (39, 7),
 (17, 7),
 (13, 7),
 (32, 7),
 (35, 7),
 (26, 7),
 (34, 6),
 (28, 6),
 (48, 6),
 (10, 6),
 (43, 6),
 (6, 6),
 (1, 5),
 (27, 5),
 (44, 5),
 (2, 5),
 (12, 5),
 (7, 5),
 (11, 4),
 (18, 4),
 (45, 4),
 (4, 4),
 (25, 4),
 (42, 4),
 (30, 4),
 (31, 3),
 (19, 3),
 (14, 3),
 (21, 3),
 (8, 3),
 (38, 3),
 (20, 3),
 (37, 3),
 (29, 2),
 (50, 2),
 (24, 2),
 (49, 2),
 (23, 1),
 (33, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD, and extracting lineage and of an RDD in a spark application. We also used transformations and actions to perform calculations across RDDs on a distributed setup. Up next, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
